# Assignment 3
## CS156 | Prof. Sterne
### Anirudh Nair

Colab Link:https://colab.research.google.com/drive/1XFkRxotPD2ualhUua2GlKQBp3byqrnMg#scrollTo=rh7RAWpHROxi (higher sample size for testing and training)

In [ ]:
#importing the required libraries
import numpy as np
from zipfile import ZipFile
from glob import glob
from PIL import Image
import PIL.ImageOps
import random as random
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
#unzipping the downloaded file and loading the jersey and shirt folders into their respective datasets
with ZipFile('jerseys (1).zip') as zipObj: zipObj.extractall()
with ZipFile('shirts (1).zip') as zipObj: zipObj.extractall()
size = (200,250)
jersey = glob("Jersey - n03595614/*")
shirt = glob("Shirt - n04197391/*")

FileNotFoundError: ignored

In [ ]:
#Standardizing all the uploaded images into one size
standardize = (200,250)

In [ ]:
# This function will be used to conver the uploaded files into a numpy array of RGB pixels
def img_to_array(filename):
    img = Image.open(filename)
    img = img.resize(standardize)
    img = list(img.getdata())
    img = np.array(img)
    img_wide = img.reshape(1, -1)
    return img_wide[0]

In [ ]:
#creating the combined dataset
combined = []

for i in jersey[0:1000]:
    combined.append([img_to_array(i), "jersey"])
    
for i in shirt[0:1000]:
    combined.append([img_to_array(i), "shirt"])

NameError: ignored

In [ ]:
#shuffling the shirts and jerseys in the combined dataset
random.seed(100)
random.shuffle(combined)

In [ ]:
#splitting the data into the training and testing sets
train = combined[:1600] 
test = combined[1600:2000]
x_train = np.array([cd for (cd,_y) in train], dtype=object) 
y_train = np.array([_y for (cd,_y) in train], dtype=object)
x_test = np.array([cd for (cd,_y) in test])
y_test = np.array([_y for (cd,_y) in test])

In [ ]:
import pandas as pd  
x_train = x_train.tolist()
y_train = y_train.tolist()

In [ ]:
#building the classifier for the data
model = LogisticRegression(solver = "saga")
model.fit(x_train, y_train)

/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
ytrain_pred = model.predict(x_train)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(model.score(x_train, y_train)))

Accuracy of logistic regression classifier on train set: 0.97


In [ ]:
ytest_pred = model.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(x_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.57


We first build a general classifier that uses logistic regression to classify the data into either a shirt or a jersey. The accuracy score on the training_set comes out to be .97 which makes sense as the model was trained using that data and there would always be some noise present in the data. However, the model does not perform so well on the training set given that it only achieves a accuracy score of .57. This shows us that weighing all elements of the image equally and using them to classify a clothing image as a shirt or jersey is not the best way. We need to focus down on some particular elements and then our accuracy might get better. 

In [ ]:
pca = PCA(n_components=7)
com_model = LogisticRegression(solver="saga")

pipe = Pipeline([('pca', pca), ('logistic', com_model)])


print('model accuracy on training set after pca: %s'% str(sum(pipe.fit(x_train, y_train).predict(x_train) == y_train)/len(y_train)))
print('model accuracy on test set after pca: %s'% str(sum(pipe.fit(x_train, y_train).predict(x_test)== y_test)/len(y_test)))

model accuracy on training set after pca: 0.6375
model accuracy on test set after pca: 0.45


However, when we run PCA, we see that the model does not even perform well on the training set, which was used to build the model. This could be explained by understanding the nature of PCA. PCA focuses on reducing dimensionality (getting rid of the unhelpful elements of the image) by focusing on the dimensions(elements) with maximum variation. This may not be a useful method to weigh and prioritize dimensions in ouur case because  shirt and jerseys could be similar in a lot of aspects such as size, which could also have the maximum variation in the data. Thus, the model proves its inefficiency by scoring a meagre 0.45 on the test set.

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=7)
pipe2 = Pipeline([('lda', lda), ('logistic', com_model)])


print('model accuracy on training set after lda: %s'% str(sum(pipe2.fit(x_train, y_train).predict(x_train) == y_train)/len(y_train)))
print('model accuracy on test set after lda: %s'% str(sum(pipe2.fit(x_train, y_train).predict(x_test)== y_test)/len(y_test)))

/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150000, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


model accuracy on training set after lda: 0.9875


/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150000, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/Users/anirudhnair/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


model accuracy on test set after lda: 0.5


In the case of LDA, we see that the model achieves an accuracy of 0.9875 on the training set, which is higher than the accuracy that the general model had in the beginning. This could also be explained by the nature of LDA, as LDA chooses to reduce dimensionality by maximizing the seperability between different elements of the image. Thus, it will weigh elements that are different for shirts and jerseys more and priotitize them. However, when it comes to the test set, this model doesn't perform that well. This could be becuase the training set may be overfitting the data. 